In [1]:
import requests
import base64
import datetime

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

I'm pasting those `client id` directly from the dev website, BUT... it would be better to retrieve them using a scrypt, so you don't have to keep'em static and visible in our notebook.

In [4]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expires = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception('You must set client_id and client_secret')
        
        client_credentials = f'{client_id}:{client_secret}'
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        
        return client_credentials_b64.decode()

    def get_token_headers(self):
        client_credentials_b64 = self.get_client_credentials()
        
        return {
            'authorization': f'Basic {client_credentials_b64}'
        }

    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()

        r = requests.post(token_url, data=token_data, headers=token_headers)

        if r.status_code not in range(200, 299):
            return False

        data = r.json()
        access_token = data['access_token']
        now = datetime.datetime.now()
        expires_in = data['expires_in']  # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expires = expires < now
        
        return True
        

In [5]:
client = SpotifyAPI(client_id, client_secret)

In [6]:
client.perform_auth()

True

In [7]:
client.access_token

'BQDv1-bEE0_M4mIEswK8AMGz6Kv7bYgIQmid6dySJ-EC3GzNI5Ugpx2DYwWmZzzxcyXAP-XB71HAO8NbxxE'